In [46]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import os
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm

In [2]:
model = load_model('model_best.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.





In [3]:
unique_breeds = pd.read_csv('encoding_label.csv')
unique_breeds = unique_breeds.drop('Unnamed: 0', axis=1)

In [37]:
breed_list = os.listdir('./dataset/test')

In [39]:
fierce_dog = ['american_staffordshire_terrier', 'japanese_mastiff', 'pitbull_terrier', 'rottweiler', 'staffordshire_bullterrier']

In [42]:
def transfer_image(img):
    resize = img.resize((224, 224))
    resize = np.array(resize)/255.
    resize = resize.reshape(1, 224, 224, 3)
    return resize

In [43]:
def get_pred_label(pred_prob):
    return unique_breeds.iloc[np.argmax(pred_prob)]['breed']

In [47]:
y_pred = []
y_true = []

for breed in tqdm(breed_list):
    path = ('./dataset/test/{}'.format(breed))
    filenames = os.listdir(path)
    for filename in filenames:
        path = ('./dataset/test/{}/{}'.format(breed, filename))
        img = Image.open(path)
        resize = transfer_image(img)

        pred_prob = model.predict(resize)
        pred_label = get_pred_label(pred_prob)
        
        y_pred.append(pred_label)
        y_true.append(breed)

  0%|          | 0/122 [00:00<?, ?it/s]

In [ ]:
'''
                             Actual
                             
                        Positive    Negative
    
Predicted    Positive     TP          FP


             Negative     FN          TN
'''

In [62]:
# 전체 혼동행렬

TP = []
FP = []
FN = []
TN = []

for breed in zip(y_pred, y_true):
    if breed[0] in fierce_dog:
        if breed[1] in fierce_dog:
            TP.append(1)
        else:
            FP.append(1)
    else:
        if breed[1] in fierce_dog:
            FN.append(1)
        else:
            TN.append(1)

In [63]:
TP = sum(TP)
FP = sum(FP)
FN = sum(FN)
TN = sum(TN)

In [64]:
Precision = TP/(TP+FP)
Negative_Predictive_value = TN/(TN+FN)
Sensitivity = TP/(TP+FN)
Specificity = TN/(TN+FP)
Accuracy = (TP+TN)/(TP+TN+FP+FN)

In [85]:
lst = ['Precision', 'Negative_Predictive_value', 'Sensitivity', 'Specificity', 'Accuracy']
for i, j in enumerate([Precision, Negative_Predictive_value, Sensitivity, Specificity, Accuracy]):
    print('{} :'.format(lst[i]), round(j, 3))

Precision : 0.937
Negative_Predictive_value : 0.997
Sensitivity : 0.957
Specificity : 0.995
Accuracy : 0.993


In [ ]:
result = []
for breed in zip(y_pred, y_true):
    if (breed[0] in fierce_dog) | (breed[1] in fierce_dog):
        result.append(breed)

In [75]:
# 맹견 혼동행렬

accuracy = []
for pred, true in result:
    if pred == true:
        accuracy.append(1)

In [78]:
accuracy = sum(accuracy) / len(result)

In [86]:
round(accuracy, 3)

0.811